In [1]:
import sys
import time
import csv
import math
import numpy as np
import pyspark
from pyspark import SparkContext, SparkConf
import json
from operator import add
import pandas as pd
import xgboost as xgb
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from node2vec import Node2Vec as n2v
import networkx as nx
from hashlib import md5
import pickle
import os
import pandas as pd
from xgboost import XGBRegressor

pd.set_option("display.max_columns", 100)
pd.set_option('display.width', 100)
pd.set_option("display.precision", 2)


from utils import create_category_md5_mapping, integrate_mapping_user_bus_cat_data, dataframe_to_rdd_dict, analyze_top_business_categories, analyze_top_categories

import json
import numpy as np
from pyspark import SparkContext, SparkConf
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

def initialize_spark_context(APP_NAME="Train: XGBModel"):
    # Spark配置项列表
    SPARK_CONF = [
        ("spark.dynamicAllocation.enabled", "true"),  # 启用动态资源分配
        ("spark.dynamicAllocation.maxExecutors", "10"),  # 最大执行器数量
        ("spark.executor.memory", "3g"),  # 每个执行器的内存
        ("spark.executor.cores", "2"),  # 每个执行器的CPU核心数
        ("spark.executor.memoryOverhead", "3000"),  # 执行器内存开销
        ("spark.driver.memory", "4g"),  # 驱动程序的内存
        ("spark.driver.maxResultSize", "2g"),  # 驱动程序的最大结果大小
        ("spark.python.worker.memory", "2g"),  # Python工作进程的内存
        ("spark.sql.shuffle.partitions", "20"),  # Shuffle操作的分区数
        ("spark.sql.sources.partitionOverWriteMode", "dynamic"),  # 分区覆写模式
        ("spark.network.timeout", "600s"),  # 网络超时设置
        ("spark.executor.heartbeatInterval", "120s"),  # 执行器心跳间隔
    ]

    # 创建Spark配置
    spark_conf = pyspark.SparkConf()
    spark_conf.setAppName(APP_NAME)
    spark_conf.setAll(SPARK_CONF)

    # 创建SparkContext
    sc = pyspark.SparkContext(conf=spark_conf)
    sc.setLogLevel("ERROR")  # 设置日志级别

    return sc

sc = initialize_spark_context()

import better_features
from better_features import FeatureProcessor, read_json_data, transform_user_data, transform_business_data, extract_review_data
from datetime import datetime #add
import time
import pandas as pd

def rdd_to_pandas(rdd):
    return pd.DataFrame(rdd.collect(), columns=rdd.first().keys())

def prepare_test_data(test_df, cluster):
    if cluster == -1:
        test_cluster_data = test_df
    else:
        test_cluster_data = test_df[test_df['Cluster'] == cluster]
        
    X_test = test_cluster_data.drop(columns=['stars', 'user_id', 'business_id'])
    y_test = test_cluster_data['stars']
    return X_test, y_test

start_time = time.time()

user_parsed_df = pd.read_csv('cache/user_df.csv') # parsed from users.json
business_parsed_df = pd.read_csv('cache/business_df.csv') # parsed from business.json
review_parsed_df = pd.read_csv('cache/review_df.csv') # parsed from business.json

train_df = pd.read_csv('cache/train_df.csv', index_col=None)
test_df = pd.read_csv('cache/test_df.csv', index_col=None)
val_df = pd.read_csv('cache/val_df.csv', index_col=None)

print("DONE!")

24/05/04 01:04:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


DONE!


In [9]:
large_xgb_params = {
    0: {'learning_rate': 0.04246778091879101, 'max_depth': 6, 'n_estimators': 491, 'subsample': 0.8689},
    2: {'learning_rate': 0.12350751460907078, 'max_depth': 3, 'n_estimators': 288, 'subsample': 0.8880},
    3: {'learning_rate': 0.08288064461501718, 'max_depth': 5, 'n_estimators': 305, 'subsample': 0.6839},
    4: {'learning_rate': 0.18299396047960448, 'max_depth': 4, 'n_estimators': 250, 'subsample': 0.9081}
}

large_catboost_params = {
    0: {'depth': 12, 'l2_leaf_reg': 18.15, 'learning_rate': 0.06229, 'n_estimators': 1486},
    2: {'depth': 6, 'l2_leaf_reg': 28.02, 'learning_rate': 0.1090, 'n_estimators': 176},
    3: {'depth': 5, 'l2_leaf_reg': 0.2358, 'learning_rate': 0.1968, 'n_estimators': 62},
    4: {'depth': 4, 'l2_leaf_reg': 0.3376, 'learning_rate': 0.1914, 'n_estimators': 46}
}

# 5, 6, 7, 8
# test_size = 0.1
medium_catboost_params = {
    5: {'depth': 3, 'l2_leaf_reg': 0.415375550656062, 'learning_rate': 0.9017553809036529}, 
    7: {'depth': 6, 'l2_leaf_reg': 18.3792029910461, 'learning_rate': 0.9372003134293689}, 
    8: {'depth': 1, 'l2_leaf_reg': 7.440376345058953, 'learning_rate': 0.4049807340854126},
    6: {'depth': 2, 'l2_leaf_reg': 1.1835517768726047, 'learning_rate': 0.38770112704134657}
}

In [10]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import ElasticNetCV

# 初始化和训练模型，返回预测结果和RMSE
def train_and_predict(X_train, y_train, X_test, y_test, model_params, model_type='xgb'):
    if model_type == 'xgb':
        model = XGBRegressor(**model_params, objective='reg:squarederror', verbosity=0)
    elif model_type == 'cb':
        model = CatBoostRegressor(**model_params, verbose=False)
    else:
        raise ValueError("Unsupported model type: " + model_type)

    model.fit(X_train, y_train)
    preds = model.predict(X_test).clip(1, 5)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return preds, rmse

# 处理数据
def prepare_data(df, drop_cols):
    df = df.drop(columns=drop_cols, errors='ignore')
    X = df.drop(columns=['stars'])
    y = df['stars']
    return X, y

# 配置和数据准备
drop_cols = ['user_id', 'business_id']  # 默认去除的列

# 为cluster 0准备数据
X_train, y_train = prepare_data(train_df[train_df['Cluster'] == 0], drop_cols)
X_test, y_test = prepare_data(test_df[test_df['Cluster'] == 0], drop_cols)

# 训练模型和预测
xgb_params = large_xgb_params[0]
cb_params = large_catboost_params[0]

xgb_preds, xgb_rmse = train_and_predict(X_train, y_train, X_test, y_test, xgb_params, model_type='xgb')
cb_preds, cb_rmse = train_and_predict(X_train, y_train, X_test, y_test, cb_params, model_type='cb')

# 组合预测结果和计算最终RMSE
best_ratio = 0.5  # 假设已经通过某种方式得到最佳权重比例
final_preds = best_ratio * cb_preds + (1 - best_ratio) * xgb_preds
final_rmse = np.sqrt(mean_squared_error(y_test, final_preds))

# 输出结果
print("XGB RMSE:", xgb_rmse)
print("CB RMSE:", cb_rmse)
print("Final RMSE:", final_rmse)

XGB RMSE: 0.982394945341761
CB RMSE: 0.9872458065362693
Final RMSE: 0.9820347044803716
